## Recall 2022-12-20

- Date $(x^{(1)},y^{(1)}), \cdots , (x^{(n)},y^{(n)})$ 是從 $y=f_{\mathtt{real}}(x)$ sample 出來的 (加上一些擾動).

- $f_{\mathtt{real}}$ 一般會是連續函數.
  - [Stone–Weierstrass theorem](https://en.wikipedia.org/wiki/Stone–Weierstrass_theorem)

- 要估計 $f_{\mathtt{real}}$:
  - 去生成 $f$ s.t. $f\approx f_{\mathtt{real}}.$
  - 用 $\widehat{y}=\mathtt{Net}(x):=f(x)$ 作為估計值.
  - $\mathtt{Net}=\mathtt{Net}(\theta)$: $\theta$ 為變量.
  - Linear model 大多時無用.
  - Let $\widehat{y}^{(i)}=\mathtt{Net}(x^{(i)}).$
  - 去 min $\frac{1}{n}\sum_{i=1}^n \ell(y^{(i)},\widehat{y}^{(i)}),$ where $\ell$ is some loss function.

- 理紅意老師講義
- 得到 DNN 的架構
  - 宇宙定理 5.1.1.4. en

## Outline of 2022-12-27

- 


- 為什麼 DNN 結構長這樣：微分好算
  - 5.3 Forward Propagation, Backward Propagation (en)
  - 很好用電腦表示(可用矩陣表示)

- 宇宙定理
  - 所以 DNN 只要參數足夠, 幾乎都會 Overfitting.

## 5.2.2 用 API 弄 NN

## 3.6 en overfitting 那些

## 3.7 en Regularization
  - 5.6 Dropout en
  - https://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2017/Lecture/DNN%20tip.pdf
  - 銀行人員？
  - 破壞細胞？

## 5.4 梯度消失 以及 W 初始化的問題



## 5.2.2 Net API

Visualize Net

## 4.9.3 分布偏移

- https://www.zhihu.com/question/26485586
- https://roger010620.medium.com/l1-l2-regularization-原理-5beb1b68b955
- https://hackmd.io/@kk6333/BkIDyLikj

- https://bobondemon.github.io/2017/11/14/Notes-for-KKT-Conditions/
- https://stats.stackexchange.com/a/348505
- https://www.zhihu.com/question/26485586